In [ ]:
pip install langchain langchain-community langchain-core langchain-openai faiss-cpu pypdf

In [43]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.conversational_retrieval import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import SystemMessage, HumanMessage


ModuleNotFoundError: No module named 'langchain.conversational_retrieval'

In [ ]:
apikeyGpt = os.environ.get('GPT_APIKEY')
if not apikeyGpt:
    raise RuntimeError("GPT_APIKEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["OPENAI_API_KEY"] = apikeyGpt

In [ ]:
__loader__ = PyPDFLoader("./arquivos/manual_avancado_nobreak_fxp2000.pdf")
documents = __loader__.load()
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

chat = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
message = """Você é um assistente técnico especializado em nobreaks da marca SMS, modelo FXP2000.
Utilize as informações do manual do usuário para responder às perguntas dos clientes.
Se a informação não estiver no manual, responda que não sabe a resposta."""

system_meessage = SystemMessage(content=message)

In [ ]:
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

qa_chain = create_retrieval_chain(
    llm=chat,
    retriever=retriever,
    return_source_documents=True
)

chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key="question",
    output_message_key="chat"
)